In [1]:
'''
this notebook demonstrates the 2d impact of changing various camera parameters.
nothing world shattering in here, but some helpers to conceptualize these connections.
managing to perfectly align the visible track lanes with the camera means you have
found the correct camera calibration for this image.
'''

%matplotlib widget

import ipywidgets as widgets
from ipywidgets import fixed
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image

import sys
sys.path.append('../src/')
from sim_util import *

img = Image.open('../figs/letesenbet_frame50.jpg')

In [8]:
plt.figure(figsize=(14, 6))
gs1 = gridspec.GridSpec(1, 2)
gs1.update(wspace=0.25, hspace=0.0, top=1., bottom=-.1)

ax = plt.subplot(gs1[0], projection='3d')
ax1 = plt.subplot(gs1[1])

param_ranges = {'azim': (-90.,90), 
                'elev': (-20., -1),
                'fov': (1, 80.),
                'rot': (-10., 10.),
                'target_x': (-20., 20.),
                'target_y': (-10., 10.),
                'dist': (20., 100)
               }

@widgets.interact(F=(1, 10), **param_ranges)
def render_scene(azim=-74.4, elev=-3.2, dist=88.4, fov=4.2, F=1, rot=0, target_x=1.71, target_y=1.62):
    ax.clear()
    ax1.clear()
        
    view_target = np.array([target_x, target_y, 0.])
    azel_vec = azim_to_vec(azim, elev, degrees=True)
    cam_vec = view_target - azel_vec * dist
    print('cam_vec', cam_vec)
    
    with plt.rc_context({'axes.edgecolor':'orange', 'xtick.color':'green', 'ytick.color':'green', 'figure.facecolor':'white'}):
        res_dict = compute_projection(camx=cam_vec[0], camy=cam_vec[1], camz=cam_vec[2], azim=azim, 
                                      _lanes=lanes_new, no_vanish=True, elev=elev, fov=fov, F=F, roll=rot)
                    
        draw_scene(res_dict, fake_F=10, ax=ax, _lanes=lanes_new)
        draw2d(ax1, res_dict['lines_2ds'], color='blue', linewidth=2, alpha=.8)
        
    ax.scatter(*view_target, marker='x', color='red')
    
    ax1.imshow(np.flip(img, 0))
    ax.set_xlim(0,50)
        
    ax1.axis('off')
    ax.set_xlim(-20, 50)
    ax.set_ylim(0, 50)
    ax.set_zlim(-20, 20)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=-74.4, description='azim', max=90.0, min=-90.0), FloatSlider(value=-3.…